## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import timeit
import time

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
WeatherPy_cities_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
WeatherPy_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ahipara,NZ,-35.1667,173.1667,62.62,74,100,4.21,overcast clouds
1,1,Qui Nhon,VN,13.7667,109.2333,86.32,66,99,8.08,moderate rain
2,2,Busselton,AU,-33.6500,115.3333,57.69,60,95,13.53,overcast clouds
3,3,Daru,PG,-9.0763,143.2092,79.38,87,68,15.21,broken clouds
4,4,Hermanus,ZA,-34.4187,19.2345,54.19,80,46,4.63,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_WeatherPy_cities_df = WeatherPy_cities_df.loc[(WeatherPy_cities_df["Max Temp"] <= max_temp) & \
                                       (WeatherPy_cities_df["Max Temp"] >= min_temp)]
preferred_WeatherPy_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Daru,PG,-9.0763,143.2092,79.38,87,68,15.21,broken clouds
5,5,Sao Francisco De Assis,BR,-29.5503,-55.1311,70.84,66,43,16.04,scattered clouds
7,7,Neiafu,TO,-18.6500,-173.9833,81.28,74,40,13.80,scattered clouds
8,8,Kaeo,NZ,-35.1000,173.7833,65.32,86,100,3.83,overcast clouds
11,11,Port Blair,IN,11.6667,92.7500,80.64,89,100,4.61,drizzle
12,12,Devils Lake,US,48.1128,-98.8651,80.58,76,40,6.91,scattered clouds
14,14,Lompoc,US,34.6391,-120.4579,70.84,82,75,18.99,mist
16,16,Chambersburg,US,39.9376,-77.6611,68.77,78,92,1.01,overcast clouds
18,18,Hamilton,US,39.1834,-84.5333,76.05,78,20,12.66,few clouds
21,21,Nabire,ID,-3.3667,135.4833,83.37,74,98,3.53,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_WeatherPy_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399 entries, 3 to 718
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              399 non-null    int64  
 1   City                 399 non-null    object 
 2   Country              399 non-null    object 
 3   Lat                  399 non-null    float64
 4   Lng                  399 non-null    float64
 5   Max Temp             399 non-null    float64
 6   Humidity             399 non-null    int64  
 7   Cloudiness           399 non-null    int64  
 8   Wind Speed           399 non-null    float64
 9   Current Description  399 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 34.3+ KB


In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows found. If found would use dropna() in filtering statement.


In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_WeatherPy_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Daru,PG,79.38,broken clouds,-9.0763,143.2092,
5,Sao Francisco De Assis,BR,70.84,scattered clouds,-29.5503,-55.1311,
7,Neiafu,TO,81.28,scattered clouds,-18.6500,-173.9833,
8,Kaeo,NZ,65.32,overcast clouds,-35.1000,173.7833,
11,Port Blair,IN,80.64,drizzle,11.6667,92.7500,


In [7]:
# 6a.1. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}
# 6.a.2 Set count and record count
record_count = 1
set_count = 1
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Set a timer so API queries don't exceed 50 per minute.
    if (index % 50 == 0 and index >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Add 1 to the record count (for purposes of timing the record count).
    record_count += 1

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [8]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Daru,PG,79.38,broken clouds,-9.0763,143.2092,Daru Lodge
5,Sao Francisco De Assis,BR,70.84,scattered clouds,-29.5503,-55.1311,Hotel Aguiar
7,Neiafu,TO,81.28,scattered clouds,-18.6500,-173.9833,Mystic Sands Beachfront Accommodation
8,Kaeo,NZ,65.32,overcast clouds,-35.1000,173.7833,Bushido Lodge
11,Port Blair,IN,80.64,drizzle,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
...,...,...,...,...,...,...,...
714,Am Timan,TD,72.54,overcast clouds,11.0297,20.2827,Palais du chef Canton Salamt
715,Pauini,BR,75.79,broken clouds,-7.7136,-66.9764,Casa do Kiwliks
716,La Rochelle,FR,69.94,clear sky,46.1667,-1.1500,Hôtel Saint Jean d'Acre
717,San Ramon,US,76.39,clear sky,37.7799,-121.9780,San Ramon Marriott


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))